<a href="https://colab.research.google.com/github/dejiandrew/nba-award-predictor/blob/deji/lightgbm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
!rm *.csv
!rm *.pkl

In [161]:
!pip install wget
import pandas as pd
import wget
import duckdb
import lightgbm as lgb
import joblib

In [162]:
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly.csv')
features_overall_weekly_df = pd.read_csv('features-overall-weekly.csv')

In [163]:
# Only train on 2021+ seasons to account for modern scoring patterns.
#features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['season'] >= 2021]
features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-12-08']
#features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-12-01']
#features_overall_weekly_df = features_overall_weekly_df[features_overall_weekly_df['week_start'] != '2025-11-24']

In [164]:
features_overall_weekly_df.columns

Index(['player_id', 'full_name', 'team', 'season', 'week', 'week_start',
       'conference', 'pow_conference', 'games_played_this_week', 'numMinutes',
       'points', 'assists', 'blocks', 'steals', 'reboundsTotal',
       'reboundsDefensive', 'reboundsOffensive', 'fieldGoalsAttempted',
       'fieldGoalsMade', 'threePointersAttempted', 'threePointersMade',
       'freeThrowsAttempted', 'freeThrowsMade', 'turnovers', 'foulsPersonal',
       'plusMinusPoints', 'wins_this_week', 'wins_vs_team_with_all_nba_player',
       'is_win_vs_over_500', 'opponent_has_all_nba', 'avg_opp_score',
       'avg_opp_winrate_prior', 'avg_opp_wins_prior', 'avg_opp_losses_prior',
       'away_games_prior', 'away_losses_prior', 'away_win_streak_prior',
       'away_wins_prior', 'home_games_prior', 'home_losses_prior',
       'home_win_streak_prior', 'home_wins_prior', 'losses_prior',
       'wins_vs_over_500_prior', 'won_player_of_the_week',
       'all_star_this_season', 'mvp_this_season',
       'all_nba_f

In [165]:
# View the class imbalance (99.4% positive class, 0.6% negative class)
print(features_overall_weekly_df['won_player_of_the_week'].value_counts())
print(features_overall_weekly_df['won_player_of_the_week'].value_counts(normalize=True))

won_player_of_the_week
0    233496
1      1396
Name: count, dtype: int64
won_player_of_the_week
0    0.994057
1    0.005943
Name: proportion, dtype: float64


In [166]:
# Drop columns that contain the answer. Also drop mvp, all-star, all-nba because they are from the future when we try to do inference during season
leakage_cols = ['pow_player_id', 'player_of_the_week', 'pow_conference',
                'all_star_this_season', 'mvp_this_season',
                'all_nba_first_team_this_season', 'all_nba_second_team_this_season',
                'all_nba_third_team_this_season']

# Set aside identifier columns
id_cols = ['player_id', 'full_name', 'team', 'season', 'week', 'week_start', 'conference']

features_overall_weekly_df_clean = features_overall_weekly_df.drop(columns=leakage_cols + id_cols)
df_encoded = pd.get_dummies(features_overall_weekly_df_clean, drop_first=True)
df_encoded

,games_played_this_week,numMinutes,points,assists,blocks,steals,reboundsTotal,reboundsDefensive,reboundsOffensive,fieldGoalsAttempted,...,breakout_score,league_pts_mean,league_pts_std,league_ast_mean,league_ast_std,league_pm_mean,league_pm_std,z_s_pts,z_s_ast,z_s_pm
0,3,101.00,61.0,3.0,5.0,1.0,35.0,29.0,6.0,43.0,...,-0.623703,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
1,2,82.00,54.0,14.0,6.0,0.0,19.0,16.0,3.0,41.0,...,-1.218188,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
2,1,11.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,...,-1.365436,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
3,2,64.00,27.0,7.0,2.0,4.0,12.0,0.0,0.0,21.0,...,-0.576144,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
4,4,41.00,15.0,2.0,2.0,0.0,5.0,4.0,1.0,16.0,...,0.643651,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234887,2,16.62,6.0,1.0,0.0,0.0,7.0,6.0,1.0,5.0,...,-0.044507,37.456404,37.847972,8.714089,10.369954,-0.66905,30.233302,-0.831125,-0.743888,-0.374784
234888,3,114.24,36.0,18.0,0.0,6.0,6.0,6.0,0.0,34.0,...,0.981656,37.456404,37.847972,8.714089,10.369954,-0.66905,30.233302,-0.038480,0.895463,0.617500
234889,2,18.74,8.0,2.0,0.0,1.0,2.0,2.0,0.0,6.0,...,0.859651,37.456404,37.847972,8.714089,10.369954,-0.66905,30.233302,-0.778282,-0.647456,-0.672469
234890,3,8.66,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,...,-0.790796,37.456404,37.847972,8.714089,10.369954,-0.66905,30.233302,-0.963233,-0.840321,-0.010947


In [167]:
from sklearn.model_selection import GroupShuffleSplit

X = df_encoded.drop(columns=['won_player_of_the_week'])
y = df_encoded['won_player_of_the_week']
identifiers = features_overall_weekly_df[id_cols]

# Create groups for each season-week-conference combination
groups = (features_overall_weekly_df['season'].astype(str) + '_' +
          features_overall_weekly_df['week'].astype(str) + '_' +
          features_overall_weekly_df['conference'].astype(str))

# Use GroupShuffleSplit instead of train_test_split
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

# Split using the indices
X_train = X.iloc[train_idx]
X_test = X.iloc[test_idx]
y_train = y.iloc[train_idx]
y_test = y.iloc[test_idx]
id_train = identifiers.iloc[train_idx]
id_test = identifiers.iloc[test_idx]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Positive class in y_train: {100 * round(y_train.mean(),4)}%")
print(f"Positive class in y_test: {100 * round(y_test.mean(),4)}%")

Training samples: 187736
Test samples: 47156
Positive class in y_train: 0.6%
Positive class in y_test: 0.59%


In [168]:
!pip install lightgbm

In [169]:

# Train LightGBM model
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    num_leaves=31,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# Train LightGBM model
lgb_model.fit(X_train, y_train)

# Get predictions
y_pred_proba = lgb_model.predict_proba(X_test)[:, 1]

In [170]:
evaluation_df = id_test.copy()
evaluation_df['actual_won'] = y_test.values
evaluation_df['pred_proba'] = y_pred_proba
evaluation_df

,player_id,full_name,team,season,week,week_start,conference,actual_won,pred_proba
1946,305,Robert Parish,Warriors,1979,51,1979-12-17,West,0,0.222927
1947,1453,Walter Davis,Suns,1979,51,1979-12-17,West,0,0.713953
1948,76003,Kareem Abdul-Jabbar,Lakers,1979,51,1979-12-17,West,0,0.939235
1949,76011,Alvan Adams,Suns,1979,51,1979-12-17,West,0,0.482696
1950,76085,James Bailey,SuperSonics,1979,51,1979-12-17,West,0,0.000779
...,...,...,...,...,...,...,...,...,...
234116,1642959,Chris Youngblood,Thunder,2025,47,2025-11-17,West,0,0.000101
234117,1642962,Drake Powell,Nets,2025,47,2025-11-17,East,0,0.000812
234118,1642964,Brooks Barnhizer,Thunder,2025,47,2025-11-17,West,0,0.000085
234119,1643007,Taelon Peter,Pacers,2025,47,2025-11-17,East,0,0.000187


In [171]:
actual_winners_df = evaluation_df[evaluation_df['actual_won'] == 1]
actual_winners_df.head()

,player_id,full_name,team,season,week,week_start,conference,actual_won,pred_proba
2016,77689,Swen Nater,Clippers,1979,51,1979-12-17,West,1,0.240036
3538,77952,Micheal Ray Richardson,Knicks,1980,12,1981-03-16,East,1,0.962766
3934,305,Robert Parish,Celtics,1980,48,1980-11-24,East,1,0.672140
4639,78549,Gus Williams,SuperSonics,1981,1,1982-01-04,West,1,0.980871
4773,76091,Greg Ballard,Bullets,1981,3,1982-01-18,East,1,0.917596


In [172]:
print(f"Total samples in evaluation_df: {len(evaluation_df)}")
print(f"Actual winners: {len(actual_winners_df)}")

Total samples in evaluation_df: 47156
Actual winners: 278


In [173]:
# How many season,conference,week combinations were there in actual_winners_df? This is our denominator for accuracy.

query = """
SELECT COUNT(DISTINCT(season,conference,week)) AS total_season_conference_weeks_we_couldve_predicted
FROM actual_winners_df
"""

total_season_conference_weeks_we_couldve_predicted = duckdb.query(query).df()['total_season_conference_weeks_we_couldve_predicted'].sum()
total_season_conference_weeks_we_couldve_predicted

np.int64(274)

In [174]:
# How many of the actual winners did we predict?
# Aka how many of the actual winners did our model assign a top-k probability for that [season,conference,week] combo?
for k in [1, 2, 3, 4, 5, 10]:
  query = f"""

  WITH Highest_Probabilities AS (
  SELECT *
  ,RANK() OVER(PARTITION BY season, conference, week ORDER BY pred_proba DESC) AS rnk
  FROM evaluation_df
  )
  ,
  Actual_Winners AS (
    SELECT * FROM evaluation_df
    WHERE actual_won = 1
  )


  SELECT * FROM Highest_Probabilities
  JOIN Actual_Winners
  ON
  Highest_Probabilities.season = Actual_Winners.season
  AND
  Highest_Probabilities.week = Actual_Winners.week
  AND
  Highest_Probabilities.conference = Actual_Winners.conference
  WHERE rnk <= {k}
  AND Highest_Probabilities.pred_proba = Actual_Winners.pred_proba
  """
  duckdb.query(query).df()
  correct_predictions_df = duckdb.query(query).df()
  total_correct_predictions = correct_predictions_df["actual_won"].sum()
  print(f"Top-{k} accuracy:", round(100*total_correct_predictions/total_season_conference_weeks_we_couldve_predicted,2),"%")

Top-1 accuracy: 50.36 %
Top-2 accuracy: 73.36 %
Top-3 accuracy: 81.02 %
Top-4 accuracy: 85.77 %
Top-5 accuracy: 88.69 %
Top-10 accuracy: 97.08 %


In [175]:
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

feature_importance['importance_pct'] = (feature_importance['importance'] / feature_importance['importance'].sum() * 100)

print("LightGBM Top 20 features:")
print(feature_importance.head(20)[['feature','importance_pct']])

LightGBM Top 20 features:
                   feature  importance_pct
2                   points        7.533333
36                team_pts        5.966667
41    fieldGoalsPercentage        4.466667
18          wins_this_week        3.866667
6            reboundsTotal        2.900000
3                  assists        2.700000
1               numMinutes        2.700000
37                team_ast        2.633333
44      points_mean_season        2.566667
14          freeThrowsMade        2.500000
61                 z_s_ast        2.466667
10          fieldGoalsMade        2.266667
50                   z_pts        2.133333
21    opponent_has_all_nba        2.033333
60                 z_s_pts        2.033333
0   games_played_this_week        2.000000
7        reboundsDefensive        1.900000
39                team_stl        1.833333
53          breakout_score        1.833333
46     assists_mean_season        1.833333


In [176]:
# import joblib

# # Save model
# joblib.dump(lgb_model, 'lightgbm_potw_model.pkl')

# # Save feature names
# joblib.dump(X_train.columns.tolist(), 'lightgbm_model_features.pkl')

In [177]:
print("\nTraining final model on 100% of data for deployment...")
final_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    num_leaves=31,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

final_model.fit(X, y)  # ALL data

# 5. Save the final model
joblib.dump(final_model, 'lightgbm_potw_model.pkl')
joblib.dump(X.columns.tolist(), 'lightgbm_model_features.pkl')


Training final model on 100% of data for deployment...


['lightgbm_model_features.pkl']

In [178]:
# Feature importance from final model
final_feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

final_feature_importance['importance_pct'] = (final_feature_importance['importance'] / final_feature_importance['importance'].sum() * 100)

print("Final Model Top 20 features:")
print(final_feature_importance.head(20)[['feature','importance_pct']])

Final Model Top 20 features:
                   feature  importance_pct
36                team_pts        6.566667
2                   points        6.300000
41    fieldGoalsPercentage        4.633333
18          wins_this_week        3.966667
6            reboundsTotal        3.033333
3                  assists        2.933333
1               numMinutes        2.900000
37                team_ast        2.766667
10          fieldGoalsMade        2.600000
7        reboundsDefensive        2.433333
44      points_mean_season        2.366667
50                   z_pts        2.366667
46     assists_mean_season        2.366667
21    opponent_has_all_nba        2.266667
61                 z_s_ast        2.233333
60                 z_s_pts        2.066667
4                   blocks        2.033333
0   games_played_this_week        2.000000
14          freeThrowsMade        1.966667
51                   z_ast        1.900000


**The below steps will use the above model for inference.**

In [179]:
def get_potw_predictions(week_start: str):
  """
  get_potw_predictions() does inference to predict POW.
  week_start must be a Monday in 'yyyy-mm-dd' format
  """
  import duckdb
  import joblib
  import wget

  # features-overall-weekly-for-inference.csv is calculated in a separate script that runs within the daily production pipeline
  # features-overall-weekly-for-inference.csv is needed to make predictions while the current week is in-progress (i.e. no POW announced yet)
  wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/features-overall-weekly-for-inference.csv')

  # Read in model & its feature names
  model = joblib.load('lightgbm_potw_model.pkl')
  feature_names = joblib.load('lightgbm_model_features.pkl')

  # Read in player-week stats
  overall_weekly_agg_df = pd.read_csv('features-overall-weekly-for-inference.csv')

  # Query all players who played for the input week
  query = f"""
  SELECT * FROM overall_weekly_agg_df
  WHERE week_start = '{week_start}'
  """
  df = duckdb.query(query).df()

  #Keep player info separate
  player_info = df[['player_id', 'full_name', 'conference', 'season', 'week']]
  X = df[feature_names]
  probabilities = model.predict_proba(X)[:, 1]

  # Attach probabilities to the player identities
  results = player_info.copy()
  results['probability'] = probabilities

  # Now partition by season, week, and conference and return top 5 for each conference
  results
  query = f"""
  WITH CTE AS (SELECT *,
  RANK() OVER(PARTITION BY season, week, conference ORDER BY probability DESC) AS rank
  FROM results
  )

  ,CTE2 AS (
  SELECT * FROM CTE WHERE conference = 'East' AND rank <= 5
  UNION ALL
  SELECT * FROM CTE WHERE conference = 'West' AND rank <= 5

  )
  SELECT
  '{week_start}' AS week_start
  ,rank
  ,full_name as name
  ,conference
  ,probability
  FROM CTE2
  """

  top_five_per_conference = duckdb.query(query).df()

  return top_five_per_conference

get_potw_predictions('2025-12-08')

,week_start,rank,name,conference,probability
0,2025-12-08,1,Jalen Brunson,East,0.656988
1,2025-12-08,2,Brandon Ingram,East,0.552405
2,2025-12-08,3,Jaylen Brown,East,0.462836
3,2025-12-08,4,Donovan Mitchell,East,0.385596
4,2025-12-08,5,Josh Giddey,East,0.345716
5,2025-12-08,1,Luka Doncic,West,0.699881
6,2025-12-08,2,Stephon Castle,West,0.663541
7,2025-12-08,3,Nikola Jokic,West,0.654168
8,2025-12-08,4,Stephen Curry,West,0.543329
9,2025-12-08,5,LeBron James,West,0.119025
